In [ ]:
from huggingface_hub import login


In [2]:
!pip install -U transformers peft bitsandbytes accelerate trl


  Using cached peft-0.17.0-py3-none-any.whl.metadata (14 kB)
  Using cached bitsandbytes-0.46.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached trl-0.21.0-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-21.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.12.15-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [ ]:
from transformers import pipeline
# use the pipeline for question - answer
pipeline_qn = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [4]:
# use qa_pipeline
context = "You are a doctor"
question = "I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.Your comments are "

pipeline_qn(text_inputs=question)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.Your comments are  helpful. I am glad I am not the only one who has experienced this. I am also worried about my future health. I hope I am going to be okay. Please do the same thing and let me know if you feel the same. I feel very confused.'}]

In [5]:
# Example: asking the model to summarize a text
text_to_summarize = "The quick brown fox jumps over the lazy dog. The fox is known for its speed, while the dog is known for its laziness. This simple sentence is often used to demonstrate a variety of linguistic concepts."

# The prompt must follow the model's expected format
prompt = f"[INST]You are a doctor. Summarize the following text: {question}[/INST]"

In [ ]:
# Run the pipeline with the formatted prompt
result = pipeline_qn(prompt, max_new_tokens=500, do_sample=True, temperature=0.7)

# The result is a list of dictionaries, so you'll need to extract the generated text
generated_text = result[0]['generated_text']

# remove all the texts between [INST] and [/INST] from the generated text. Use string index to Find [INST] and [/INST] and remove all the caharacters in between these including [INST] abd [/INST]
start_index = generated_text.find("[INST]")
end_index = generated_text.find("[/INST]")
print(generated_text[end_index+7:])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


I'm not sure what to make of your description. It sounds like you're experiencing symptoms of vertigo, which can be caused by a variety of factors, including inner ear problems, medication side effects, or even motion sickness.

Vertigo is a condition where you feel like you're spinning or feeling off-balance, even when you're not actually moving around. This can be quite disorienting and uncomfortable. The good news is that vertigo is usually treatable with some simple lifestyle changes, such as avoiding triggers, taking medication, or practicing relaxation techniques.

Here are some possible explanations for your symptoms:

1. **Inner ear problems**: You may have an inner ear disorder, such as benign paroxysmal positional vertigo (BPPV), labyrinthitis, or vestibular neuritis, which can cause vertigo.
2. **Medication side effects**: Certain medications, such as antibiotics, antihistamines, or antidepressants, can cause vertigo as a side effect.
3. **Motion sickness**: You may be exper

In [4]:
from datasets import load_dataset

ds = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")

In [5]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

# def format_data(example):
#     context = f"{example['instruction']} {example['input']}\n doctor: {example['output']}"
#     return tokenizer(context, truncation=True, return_tensors="pt")

# tokenizer.pad_token = tokenizer.eos_token
# formatted_data = ds.map(format_data, remove_columns=ds["train"].column_names)


from datasets import load_dataset


# Example formatting function
def format_data_for_llama(row):
    # This is a common template for Llama instruct models.
    # The 'input' field is optional and can be an empty string if not provided.
    prompt = f"[INST] {row['instruction']}\n{row['input']} [/INST]"
    response = f"{row['output']}"
    # Combine the prompt and response for the model to learn the full sequence.
    return {"text": prompt + response}

# Apply the formatting to the entire dataset
formatted_dataset = ds.map(format_data_for_llama)

In [7]:
formatted_dataset['train']

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 112165
})

In [5]:
# Set up quantization config for 4-bit loading (saves GPU memory)
# This is crucial for running larger models on limited hardware.
# from bitsandbytes import BitsAndBytesConfig
from transformers import TrainingArguments
from peft import LoraConfig

from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",  
    bnb_4bit_compute_dtype=torch.bfloat16, 
    bnb_4bit_use_double_quant=True,  # ✅ Enable double quant for more VRAM savings
)


# LoRA configuration (Parameter-Efficient Fine-Tuning)
# LoRA injects small, trainable matrices into the model, making fine-tuning
# much more efficient than updating all model parameters.
lora_config = LoraConfig(
    r=4, # Rank of the update matrices (adjust based on experimentation)
    lora_alpha=16, # Scaling factor for LoRA
    target_modules=["q_proj", "v_proj"], # Modules to apply LoRA to
    lora_dropout=0.05, # Dropout probability for LoRA layers
    bias="none", # Do not train bias terms
    task_type="CAUSAL_LM", # Specify the task type
)

# Training arguments
# These control the training process, including learning rate, epochs, etc.
training_arguments = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=1,         # minimal batch
    gradient_accumulation_steps=32,        # reduce from 64 if you want faster step time
    warmup_steps=2,
    max_steps=50,
    optim="paged_adamw_8bit",
    save_strategy="epoch",
    logging_steps=200,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
    push_to_hub=False,
    dataloader_num_workers=2,
    gradient_checkpointing=True,           # << reduces VRAM usage
)


# training_args = TrainingArguments(
#     output_dir="./results",
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=4,
#     learning_rate=2e-4,
#     max_grad_norm=0.3,
#     max_steps=500,
#     logging_steps=10,
#     fp16=True
# )

In [8]:
formatted_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 112165
    })
})

In [6]:
# reduce the dataset to 1000 samples for faster training
formatted_dataset = formatted_dataset['train'].shuffle(seed=42).select(range(1000))

In [7]:
formatted_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 1000
})

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer
import torch

model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-3.2-1B-Instruct',
    quantization_config=bnb_config,
    device_map="auto", # Automatically maps model to available devices (GPU/CPU)
)

In [11]:
# what is the model size after quantization
model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model size after quantization: {model_size / 1e6:.2f} million parameters")  


Model size after quantization: 262.74 million parameters


In [9]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
print("Initializing SFTTrainer...")
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    peft_config=lora_config,
    args=training_arguments
)



Initializing SFTTrainer...


In [14]:
torch.cuda.empty_cache()

In [ ]:

print("Starting training...")
trainer.train()
print("Training complete!")


In [12]:
trainer.save_model("health_assistant_model")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# load the model from the saved directory
model = AutoModelForCausalLM.from_pretrained("./model/health_assistant_model")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
# Example usage of the fine-tuned model
pipeline_qn = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)


/media/sudip/linux-extra/code/llm-fine-tune/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cuda:0


: 

In [1]:
# wrap the above code in a function
def ask_health_expert(question,pipeline):    
    prompt = f"[INST]You are a doctor. Generate answer to my question.My question: {question} . Your answer is : [/INST]"
    result = pipeline(prompt, max_new_tokens=500, do_sample=True, temperature=0.7)
    generated_text = result[0]['generated_text']
    start_index = generated_text.find("[INST]")
    end_index = generated_text.find("[/INST]")
    generated_txt=generated_text[end_index+7:]  # Return the answer after the [/INST] tag

    # Remove any [INST] or [/INST] od "[" , "]"
    generated_txt = generated_txt.replace("[INST]", "").replace("[/INST]", "").replace("[","").replace("]","").strip()
    return generated_txt





In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Load base model
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # e.g., "meta-llama/Llama-2-7b-hf"
base_model = AutoModelForCausalLM.from_pretrained(model_name)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load PEFT adapter
peft_model_id = "./model/health_assistant_model"  # path where you saved with save_model()
model_peft = PeftModel.from_pretrained(base_model, peft_model_id)

# Merge adapter into base model (optional but can improve performance)
# model = model.merge_and_unload()

# Set to evaluation mode
model_peft.eval()

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model_peft.to(device)

/media/sudip/linux-extra/code/llm-fine-tune/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=4, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=4, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_fea

In [3]:
from transformers import pipeline

pipeline_qn_peft = pipeline(
    "text-generation",
    model=model_peft,
    tokenizer=tokenizer
)


Device set to use cuda:0


In [4]:
question = "I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.Your comments are "

result = ask_health_expert(question,pipeline_qn_peft)

# structure the long text output from the model
def structure_output(text):
    # Split the text into sentences or paragraphs
    sentences = text.split('. ')
    structured_output = []
    
    for sentence in sentences:
        # Clean up each sentence
        cleaned_sentence = sentence.strip()
        if cleaned_sentence:  # Only add non-empty sentences
            structured_output.append(cleaned_sentence)
    
    return structured_output

structured_result_1 = structure_output(result)

for idx, sentence in enumerate(structured_result_1, 1):
    print(f"{sentence}") 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hi there, I can see that you are experiencing symptoms of vertigo, which is a sensation of spinning or dizziness
Vertigo is a type of balance disorder that can be caused by a variety of factors, including inner ear problems, neurological disorders, or even medication side effects
The fact that you are experiencing symptoms of vertigo even after taking Panadol (pantoprazole) and sleeping for a few hours suggests that the issue may not be related to your medication
Panadol is an antacid that can help alleviate nausea and vomiting, but it is not a treatment for vertigo
Vertigo is typically caused by a problem with the inner ear, such as a vestibular disorder, which can be caused by a variety of factors, including inner ear problems, neurological disorders, or even medication side effects
It is also possible that you may be experiencing motion sickness, which is a common cause of vertigo
If you are experiencing symptoms of vertigo, I would recommend consulting an ENT specialist or a neurol

In [5]:
# create a gradio interface for the health expert model
import gradio as gr 
def gradio_interface(question):
    answer = ask_health_expert(question, pipeline_qn_peft)
    structured_answer = structure_output(answer)
    return structured_answer

gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Ask a health-related question"),
    outputs=gr.Text(label="Structured Answer"),
    title="Health Expert LLM",
    description="Ask a health-related question and get answers from the fine-tuned LLM."
).launch(share=True)  # Set share=True to create a public link
# Note: You can set share=False if you want to run it locally without a public link 


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://f1bc8268b8e6091289.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [6]:
message = [ {"role": "system", "content": "You are a health expert. Answer the user's question in detail."},
            {"role": "user", "content": "I woke up this morning feeling the whole room is spinning when i was sitting down. "
            "I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. "
            "After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, "
            "only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, "
            "the spinning lessen so i am not sure whether its connected or coincidences."}
]

result = pipeline_qn_peft(message, max_new_tokens=1000, do_sample=True, temperature=0.7)
# Print the generated response
result


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': [{'role': 'system',
    'content': "You are a health expert. Answer the user's question in detail."},
   {'role': 'user',
    'content': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.'},
   {'role': 'assistant',
    'content': "I can help you try to understand what might be going on. The symptoms you're describing, such as the spinning sensation when you're sitting down, feeling nauseous and vomiting, and the spinning sensation when you're lying down, ar

In [7]:
def structure_output_from_result(result):
    for output in result[0]['generated_text'] :
        if output['role'] == 'assistant':
            return output['content'].strip()

def call_llm_experts(input_text, pipeline=pipeline_qn_peft):

    message = [{"role": "user", "content": input_text}]
    message = [ {"role": "system", "content": "You are a health expert. Answer the user's question in detail."}] + message
    print("Calling LLM with message:", message)
    result = pipeline(message, max_new_tokens=1000, do_sample=True, temperature=0.7)
    return structure_output_from_result(result)





In [8]:
# create a gradio interface for the health expert model
import gradio as gr 
def gradio_interface(question):
    answer = call_llm_experts(question, pipeline_qn_peft)
    return answer

gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Ask a health-related question"),
    outputs=gr.Text(label="Structured Answer"),
    title="Health Expert LLM",
    description="Ask a health-related question and get answers from the fine-tuned LLM."
).launch(share=True)  # Set share=True to create a public link
# Note: You can set share=False if you want to run it locally without a public link 

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://9be5372e88907cf2ba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Calling LLM with message: [{'role': 'system', 'content': "You are a health expert. Answer the user's question in detail."}, {'role': 'user', 'content': 'i am feeling very happy always'}]


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Calling LLM with message: [{'role': 'system', 'content': "You are a health expert. Answer the user's question in detail."}, {'role': 'user', 'content': 'i feel more happiness when someone else is sad'}]
